In [ ]:

from qiskit import QuantumCircuit, transpile, assemble
from qiskit.circuit.library import RealAmplitudes
from qiskit_machine_learning.algorithms import ObjectiveFunction
import numpy as np
from sklearn.model_selection import train_test_split
from numba import jit, prange
import json
import matplotlib.pyplot as plt


# Preparing data
num_qubits = 6
matrix_size = 2**num_qubits
num_repetitions = 1000
Mz_values = np.zeros((num_repetitions, matrix_size))

eigenvalues = []
eigenvectors = []

@jit(nopython=True, parallel=True)
def calculate_H_value(num_qubits, matrix_size, K, J, g, h):
    matrix = np.zeros((matrix_size, matrix_size))
    for i in prange(matrix_size):
        for j in prange(matrix_size):
            H_value = 0
            for i1 in prange(num_qubits):
                for j1 in prange(num_qubits):
                    if j == i ^ (2 ** i1 + 2 ** j1):
                        H_value += K[i1, j1]
                    if j == i:
                        sign = 1
                        if (i & 2 ** i1) != 0:
                            sign = -sign
                        if (i & 2 ** j1) != 0:
                            sign = -sign
                        H_value += sign * J[i1, j1]

            for i1 in prange(num_qubits):
                if j == i ^ (2 ** i1):
                    H_value += g[i1]

            if j == i:
                H_value += h
            matrix[i, j] = H_value
    return matrix

@jit(nopython=True, parallel=True)
def calculate_Mzt(eigenvectors_matrix, num_qubits):
    Mzt = np.zeros(eigenvectors_matrix.shape[1])
    for column in prange(eigenvectors_matrix.shape[1]):
        i = 0
        Mz = 0
        for component in eigenvectors_matrix[column, :]:
            for n_prime in prange(num_qubits):
                if (2 ** n_prime) & i != 0:
                    Mz += abs(component) ** 2 / num_qubits
            i += 1
        Mzt[column] = 2 * Mz - 1
    return Mzt

for repetition in range(num_repetitions):
    np.random.seed(repetition)
    J = np.random.uniform(low=-1, high=0, size=(num_qubits, num_qubits))
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                J[i, j] = 0
    J = (J + J.T)

    np.random.seed(repetition + 10)
    K = np.random.uniform(low=-1, high=1, size=(num_qubits, num_qubits))
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                K[i, j] = 0
    K = (K + K.T)

    np.random.seed(repetition + 20)
    h = np.random.uniform(low=-0.04, high=0.04)

    np.random.seed(repetition + 30)
    g = np.random.uniform(low=-6, high=6, size=num_qubits)

    # Create H matrix
    matrix = calculate_H_value(num_qubits, matrix_size, K, J, g, h)

    eigenval, eigenvect = np.linalg.eigh(matrix)

    min_eigenval = np.min(eigenval)
    min_eigenvec = eigenvect[:, np.argmin(eigenval)]

    eigenvalues.append(min_eigenval)
    eigenvectors.append(min_eigenvec)

eigenvectors_matrix = np.column_stack(eigenvectors)

Mzt = calculate_Mzt(eigenvectors_matrix, num_qubits)
Mzt_row = np.array(Mzt)

# Convert data to TensorFlow tensors
features = np.array(eigenvectors)
labels = np.array(Mzt_row)
print(features)
print(Mzt_row)
#=================================

# Split data into train and test first
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Then split train further into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Print shapes to verify
print(f"Train data shapes: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"Validation data shapes: X_val {X_val.shape}, y_val {y_val.shape}")
print(f"Test data shapes: X_test {X_test.shape}, y_test {y_test.shape}")
#===============================================

import pstats
import cProfile
import json
import numpy as np
from IPython.display import clear_output
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.utils import algorithm_globals

algorithm_globals.random_seed = 12345
from qiskit import QuantumCircuit, ClassicalRegister
from qiskit.quantum_info import Statevector, SparsePauliOp
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt

def conv_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    target.cx(1, 0)
    target.rz(np.pi / 2, 0)
    return target

params = ParameterVector("θ", length=3)
circuit = conv_circuit(params)
circuit.draw(style="clifford")

def conv_layer(num_qubits, param_prefix):
    qc = QuantumCircuit(num_qubits, name="Convolutional Layer")
    qubits = list(range(num_qubits))
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits * 3)
    for q1, q2 in zip(qubits[0::2], qubits[1::2]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3
    for q1, q2 in zip(qubits[1::2], qubits[2::2] + [0]):
        qc = qc.compose(conv_circuit(params[param_index : (param_index + 3)]), [q1, q2])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, qubits)
    return qc

circuit = conv_layer(4, "θ")
circuit.decompose().draw(style="clifford")

def pool_circuit(params):
    target = QuantumCircuit(2)
    target.rz(-np.pi / 2, 1)
    target.cx(1, 0)
    target.rz(params[0], 0)
    target.ry(params[1], 1)
    target.cx(0, 1)
    target.ry(params[2], 1)
    return target

params = ParameterVector("θ", length=3)
circuit = pool_circuit(params)
circuit.draw(style="clifford")

def pool_layer(sources, sinks, param_prefix):
    num_qubits = len(sources) + len(sinks)
    qc = QuantumCircuit(num_qubits, name="Pooling Layer")
    param_index = 0
    params = ParameterVector(param_prefix, length=num_qubits // 2 * 3)
    for source, sink in zip(sources, sinks):
        qc = qc.compose(pool_circuit(params[param_index : (param_index + 3)]), [source, sink])
        qc.barrier()
        param_index += 3

    qc_inst = qc.to_instruction()

    qc = QuantumCircuit(num_qubits)
    qc.append(qc_inst, range(num_qubits))
    return qc

sources = [0, 1]
sinks = [2, 3]
circuit = pool_layer(sources, sinks, "θ")
circuit.decompose().draw(style="clifford")

class AdamOptimizer:
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None
        self.v = None
        self.t = 0

    def update_params(self, gradient, num_params):
        if self.m is None:
            self.m = np.zeros(num_params)
            self.v = np.zeros(num_params)

        self.t += 1
        self.m = self.beta1 * self.m + (1 - self.beta1) * gradient
        self.v = self.beta2 * self.v + (1 - self.beta2) * (gradient ** 2)

        m_hat = self.m / (1 - self.beta1 ** self.t)
        v_hat = self.v / (1 - self.beta2 ** self.t)

        delta_params = -self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)
        return delta_params

class QuantumCNNModel:
    def __init__(self, num_qubits, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.num_qubits = num_qubits
        self.circuit, self.ansatz = self.build_cnn(num_qubits)
        self.optimizer = AdamOptimizer(learning_rate, beta1, beta2, epsilon)

        self.weight_params = list(self.ansatz.parameters)
        self.initial_point = np.random.random(len(self.weight_params))
        self.parameter_bindings = {param: self.initial_point[param_index] for param_index, param in enumerate(self.weight_params)}
        self.bound_circuit = self.ansatz.assign_parameters(self.parameter_bindings)

    def build_cnn(self, num_qubits):
        ansatz = QuantumCircuit(num_qubits, name="CNN")
        for i in reversed(range(num_qubits.bit_length() - 1)):
            sources = list(range(2 ** i))
            sinks = list(range(2 ** i, 2 ** (i + 1)))
            ansatz.compose(conv_layer(2 ** (i + 1), f"c{i+1}"), list(range(2 ** (i + 1))), inplace=True)
            ansatz.compose(pool_layer(sources, sinks, f"p{i+1}"), list(range(2 ** (i + 1))), inplace=True)

        circuit = QuantumCircuit(num_qubits)
        circuit.compose(ansatz, range(num_qubits), inplace=True)
        return circuit, ansatz

    def forward(self, features, params, num_qubits):
        outputs = []
        for repetition in range(len(features)):
            state_vector = features[repetition]
            qc = QuantumCircuit(num_qubits)
            qc.initialize(state_vector, range(num_qubits))
            bound_ansatz = self.ansatz.assign_parameters(params)
            qc.compose(bound_ansatz, inplace=True)
            state = Statevector.from_instruction(qc)
            output = np.real(state.expectation_value(SparsePauliOp.from_list([("Z" * num_qubits, 1)])))
            outputs.append(output)
        return outputs


    def mse_loss(self, output, labels):
        num_entries = len(output)
        loss = np.sum(np.power((labels - output), 2))
        return loss / num_entries

    def mse_loss(self, output, labels):
        loss = np.mean((labels - output) ** 2)
        return loss


    def compute_gradient(self, features, labels):
        gradient = np.zeros(len(self.ansatz.parameters))
        shift = np.pi / 4  # Parameter-shift rule shift

        for param_index, param in enumerate(self.ansatz.parameters):
            original_params = self.parameter_bindings.copy()

            # Compute forward shift
            shifted_params_plus = original_params.copy()
            shifted_params_plus[param] += shift
            output_plus = self.forward(features, shifted_params_plus, self.num_qubits)
            loss_plus = self.mse_loss(output_plus, labels)

            # Compute backward shift
            shifted_params_minus = original_params.copy()
            shifted_params_minus[param] -= shift
            output_minus = self.forward(features, shifted_params_minus, self.num_qubits)
            loss_minus = self.mse_loss(output_minus, labels)

            # Parameter-shift rule formula
            gradient[param_index] = (loss_plus - loss_minus) / (2 * np.sin(shift))

        return gradient

    def train(self, X_train, y_train, X_val, y_val, num_epochs):
        train_losses = []
        val_losses = []
        for epoch in range(num_epochs):
            train_output = self.forward(X_train, self.parameter_bindings, self.num_qubits)
            train_loss = self.mse_loss(train_output, y_train)
            train_losses.append(train_loss)

            val_output = self.forward(X_val, self.parameter_bindings, self.num_qubits)
            val_loss = self.mse_loss(val_output, y_val)
            val_losses.append(val_loss)

            gradients = self.compute_gradient(X_train, y_train)
            delta_params = self.optimizer.update_params(gradients, len(self.ansatz.parameters))

            for param_index, param in enumerate(self.ansatz.parameters):
                self.parameter_bindings[param] += delta_params[param_index]

            self.bound_circuit = self.ansatz.assign_parameters(self.parameter_bindings)

            print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Plotting the training and validation losses
        plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss', marker='o')
        plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss', marker='o')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss over Epochs')
        plt.legend()
        plt.grid(True)
        plt.show()

    def print_best_parameters(self):
        if self.best_params is None:
            print("No best parameters found.")
            return

        print("Best Parameters:")
        for param_index, (param, value) in enumerate(self.best_params.items()):
            layer = None
            for i in range(self.num_qubits.bit_length() - 1):
                if f"c{i+1}" in param:
                    layer = f"Convolutional Layer {i+1}"
                    break
                elif f"p{i+1}" in param:
                    layer = f"Pooling Layer {i+1}"
                    break
            print(f"{param} (in {layer}): {value}")

    def evaluate(self, X_data, y_data):
        outputs = self.forward(X_data, self.parameter_bindings, self.num_qubits)
        loss = self.mse_loss(outputs, y_data)
        return loss

    def predict(self, X_data):
        predictions = self.forward(X_data, self.parameter_bindings, self.num_qubits)
        return predictions

# Main function to initialize and train the QuantumCNNModel
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

def main():
    num_epochs = 350
    model = QuantumCNNModel(num_qubits)

    #Training model
    model.train(X_train, y_train, X_val, y_val, num_epochs)

    # Evaluate on test set
    test_loss = model.evaluate(X_test, y_test)
    print(f"Test Loss: {test_loss:.4f}")

    # Predictions on test set
    predictions = model.predict(X_test)

    # Compare predictions and actual values
    for i in range(len(predictions)):
        print(f"Sample {i + 1} - Actual: {y_test[i]}, Predicted: {predictions[i]}")

    # Convert predictions and y_test to NumPy arrays for plotting
    predictions = np.array(predictions)
    y_test_np = np.array(y_test)

    # Calculate mean squared error
    mse = mean_squared_error(y_test_np, predictions)
    print(f"Mean Squared Error on Test Set: {mse:.4f}")


if __name__ == "__main__":
    main()

